In [86]:
import json
import ast
import pandas as pd
import os
import regex

In [87]:
import warnings
warnings.filterwarnings("ignore")

### Read comments from json files

In [88]:
# Define the directory path
directory = "data/reviews"

# Initialize an empty list to store the data
data = []

# Iterate over all files in the directory
for filename in os.listdir(directory):
    if filename.endswith(".json"):
        file_path = os.path.join(directory, filename)
        # Read the JSON file and append its contents to the data list
        with open(file_path, "r") as file:
            file_content = file.read()
            try:
                # Try to parse the content as JSON
                json_data = json.loads(file_content)
                data.append(json_data)
            except json.JSONDecodeError:
                try:
                    # If parsing as JSON fails, try to evaluate the content as a Python dictionary
                    dict_data = ast.literal_eval(file_content)
                    # Convert the dictionary to a JSON string
                    json_data = json.dumps(dict_data)
                    data.append(json.loads(json_data))
                except (ValueError, SyntaxError) as e:
                    print(f"Error parsing JSON from file {file_path}: {e}")

# Create a dataframe from the data list
df = pd.DataFrame(data)

### Remove Unnecessary Columns

In [89]:
df.columns

Index(['id', 'title', 'content', 'status', 'thank_count', 'score', 'new_score',
       'customer_id', 'comment_count', 'rating', 'images', 'thanked',
       'created_at', 'created_by', 'suggestions', 'attributes',
       'product_attributes', 'spid', 'is_photo', 'seller', 'product_id',
       'vote_attributes', 'delivery_rating', 'timeline'],
      dtype='object')

In [90]:
kept_cols_lv1 = ['id', 'title', 'content', 'status', 'rating', 
             'suggestions', 'vote_attributes', 'delivery_rating']

df = df[kept_cols_lv1]
df = df[df.status == 'approved']
df = df[df.content != ""]

kept_cols_lv2 = ['id', 'title', 'content', 'rating']
df = df[kept_cols_lv2]

In [91]:
df

,id,title,content,rating
0,9724102,Cực kì hài lòng,Gôd,5
1,19582868,Cực kì hài lòng,Hàng tốt,5
2,17441069,Cực kì hài lòng,Bàn phím chất lượng như hình ảnh. Độ nãy tốt.\...,5
3,5373697,Rất không hài lòng,bàn phím loạn xị ngậu lên . chuột thì k sao cò...,1
4,2922141,Rất tốt,"Hàng đẹp và chất lượng tốt, thuận tiện khi đi ...",5
...,...,...,...,...
7316,19856146,Cực kì hài lòng,Âm thanh tốt,5
7317,15249248,Cực kì hài lòng,"Đã nhận dc đồng hồ,Tiki giao hàng và tư vấn ch...",5
7320,14210789,Cực kì hài lòng,"Chất lượng xem ra không được như mong đợi, tuy...",3
7321,19600851,Cực kì hài lòng,Âm thanh tốt. Ko bị đau tai. Chống ổn tốt.,5


#### Preprocess data

In [92]:
from utils.preprocess import (
    remove_HTML,
    convert_unicode,
    standardize_sentence_typing,
    normalize_acronyms, 
    # word_segmentation, # When use PhoBERT
    remove_unnecessary_characters,
    remove_redundant_newlines
) 

In [93]:
def text_preprocess(text):
    text = remove_HTML(text)
    text = convert_unicode(text) 
    text = standardize_sentence_typing(text)
    text = normalize_acronyms(text)
    # text = word_segmentation(text) # When use PhoBERT
    text = remove_unnecessary_characters(text)
    text = remove_redundant_newlines(text)
    # return text.lower()
    return text

In [94]:
df['processed_content'] = df['content'].apply(text_preprocess)

In [95]:
output_cols = ['id', 'title', 'content', 'processed_content', 'rating']
df = df[output_cols]
df

,id,title,content,processed_content,rating
0,9724102,Cực kì hài lòng,Gôd,gôd,5
1,19582868,Cực kì hài lòng,Hàng tốt,hàng tốt,5
2,17441069,Cực kì hài lòng,Bàn phím chất lượng như hình ảnh. Độ nãy tốt.\...,bàn phím chất lượng như hình ảnh độ nãy tốt xứ...,5
3,5373697,Rất không hài lòng,bàn phím loạn xị ngậu lên . chuột thì k sao cò...,bàn phím loạn xị ngậu lên chuột thì không sao ...,1
4,2922141,Rất tốt,"Hàng đẹp và chất lượng tốt, thuận tiện khi đi ...",hàng đẹp và chất lượng tốt thuận tiện khi đi x...,5
...,...,...,...,...,...
7316,19856146,Cực kì hài lòng,Âm thanh tốt,âm thanh tốt,5
7317,15249248,Cực kì hài lòng,"Đã nhận dc đồng hồ,Tiki giao hàng và tư vấn ch...",đã nhận được đồng hồ tiki giao hàng và tư vấn ...,5
7320,14210789,Cực kì hài lòng,"Chất lượng xem ra không được như mong đợi, tuy...",chất lượng xem ra không được như mong đợi tuy ...,3
7321,19600851,Cực kì hài lòng,Âm thanh tốt. Ko bị đau tai. Chống ổn tốt.,âm thanh tốt không bị đau tai chống ổn tốt,5


Remove duplicated contents

In [96]:
df[df['processed_content'] == 'tốt']

,id,title,content,processed_content,rating
10,19569865,Cực kì hài lòng,Good,tốt,5
74,19272226,Cực kì hài lòng,Good,tốt,5
87,13585314,Cực kì hài lòng,tốt,tốt,5
103,15405398,Cực kì hài lòng,Tốt,tốt,5
115,7873955,Hài lòng,Good,tốt,4
...,...,...,...,...,...
7041,14283156,Cực kì hài lòng,Tốt,tốt,5
7114,19736712,Cực kì hài lòng,Good,tốt,5
7116,19810303,Cực kì hài lòng,Tốt,tốt,5
7284,9529136,Cực kì hài lòng,tốt,tốt,5


In [97]:
df.drop_duplicates(subset='processed_content', inplace=True)

In [98]:
df[df['processed_content'] == 'tốt']

,id,title,content,processed_content,rating
10,19569865,Cực kì hài lòng,Good,tốt,5


### Review label distribution

In [99]:
df['rating'].value_counts()

rating
5    3039
1     747
4     584
3     315
2     244
Name: count, dtype: int64

In [100]:
df.loc[:, 'sentiment_3_cls'] = df['rating'].apply(lambda x: 'neutral' if x == 3 else 'negative' if x < 3 else 'positive')
df.loc[:, 'sentiment_2_cls'] = df['rating'].apply(lambda x: 'negative' if x < 3 else 'positive')

In [101]:
df

,id,title,content,processed_content,rating,sentiment_3_cls,sentiment_2_cls
0,9724102,Cực kì hài lòng,Gôd,gôd,5,positive,positive
1,19582868,Cực kì hài lòng,Hàng tốt,hàng tốt,5,positive,positive
2,17441069,Cực kì hài lòng,Bàn phím chất lượng như hình ảnh. Độ nãy tốt.\...,bàn phím chất lượng như hình ảnh độ nãy tốt xứ...,5,positive,positive
3,5373697,Rất không hài lòng,bàn phím loạn xị ngậu lên . chuột thì k sao cò...,bàn phím loạn xị ngậu lên chuột thì không sao ...,1,negative,negative
4,2922141,Rất tốt,"Hàng đẹp và chất lượng tốt, thuận tiện khi đi ...",hàng đẹp và chất lượng tốt thuận tiện khi đi x...,5,positive,positive
...,...,...,...,...,...,...,...
7314,19019716,Rất không hài lòng,vầy là sao thế shop. giao nhằn hay lừa đảo thế...,vầy là sao thế shop giao nhằn hay lừa đảo thế,1,negative,negative
7317,15249248,Cực kì hài lòng,"Đã nhận dc đồng hồ,Tiki giao hàng và tư vấn ch...",đã nhận được đồng hồ tiki giao hàng và tư vấn ...,5,positive,positive
7320,14210789,Cực kì hài lòng,"Chất lượng xem ra không được như mong đợi, tuy...",chất lượng xem ra không được như mong đợi tuy ...,3,neutral,positive
7321,19600851,Cực kì hài lòng,Âm thanh tốt. Ko bị đau tai. Chống ổn tốt.,âm thanh tốt không bị đau tai chống ổn tốt,5,positive,positive


In [102]:
df['sentiment_3_cls'].value_counts()

sentiment_3_cls
positive    3623
negative     991
neutral      315
Name: count, dtype: int64

In [103]:
df['sentiment_3_cls'].value_counts(normalize=True) * 100

sentiment_3_cls
positive    73.503753
negative    20.105498
neutral      6.390749
Name: proportion, dtype: float64

### Split train, valid, test

In [106]:
from sklearn.model_selection import train_test_split

# Split the dataset into train and test sets
train_df, test_df = train_test_split(df, test_size=0.2, random_state=42)

# Split the train set into train and validation sets
train_df, valid_df = train_test_split(train_df, test_size=0.2, random_state=42)

# Print the shapes of the resulting datasets
print("Train set shape:", train_df.shape)
print("Validation set shape:", valid_df.shape)
print("Test set shape:", test_df.shape)

Train set shape: (3154, 7)
Validation set shape: (789, 7)
Test set shape: (986, 7)


In [109]:
# Calculate the sentiment_3_cls distribution for each set
train_distribution = train_df['sentiment_3_cls'].value_counts()
valid_distribution = valid_df['sentiment_3_cls'].value_counts()
test_distribution = test_df['sentiment_3_cls'].value_counts()

# Calculate the percentage distribution for each set
train_percentage = train_df['sentiment_3_cls'].value_counts(normalize=True) * 100
valid_percentage = valid_df['sentiment_3_cls'].value_counts(normalize=True) * 100
test_percentage = test_df['sentiment_3_cls'].value_counts(normalize=True) * 100

# Create a single dataframe to compare the distributions
comparison_df = pd.concat([train_distribution, train_percentage, valid_distribution, valid_percentage, test_distribution, test_percentage], axis=1)
comparison_df.columns = ['Train Set (Count)', 'Train Set (%)', 'Validation Set (Count)', 'Validation Set (%)', 'Test Set (Count)', 'Test Set (%)']
comparison_df = comparison_df.fillna(0)

# Display the comparison dataframe
comparison_df


,Train Set (Count),Train Set (%),Validation Set (Count),Validation Set (%),Test Set (Count),Test Set (%)
sentiment_3_cls,,,,,,
positive,2340,74.191503,568,71.989861,715,72.515213
negative,622,19.720989,163,20.659062,206,20.892495
neutral,192,6.087508,58,7.351077,65,6.592292


In [107]:
train_df.to_csv('processed_data/review_electronic_devices_train.csv', index=False)
valid_df.to_csv('processed_data/review_electronic_devices_valid.csv', index=False)
test_df.to_csv('processed_data/review_electronic_devices_test.csv', index=False)